In [3]:
from glove import Corpus, Glove
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import datetime,time
from flashtext.keyword import KeywordProcessor
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
import pandas as pd
import numpy as np
import string
import multiprocessing
from nltk.corpus import stopwords
from flashtext.keyword import KeywordProcessor
from sklearn.model_selection import train_test_split
from sklearn import metrics

# libraries for dataset preparation, feature engineering, model training 
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
#from utils.language_models import WordVectorEmbedding
from gensim.test.utils import datapath, get_tmpfile
#import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from sklearn import preprocessing
from pymongo import MongoClient

# Import Translator module from googletrans package
from googletrans import Translator  

C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [1]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
DATABASE_NAME='Portfolio'

In [4]:
client_prod = MongoClient(host=MONGO_HOST,port=MONGO_PORT,unicode_decode_error_handler='ignore')
db_prod = client_prod[DATABASE_NAME]
collection_name = "Election_2019"
collection=db_prod[collection_name]
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, unicode_decode_error_handler='ignore'), 'Portfolio'), 'Election_2019')

In [5]:
df_tweets=pd.DataFrame(list(collection.find({})))
print('Total number of tweets loaded from Mongo : {}'.format(df_tweets.shape[0]))
df_tweets.head()

Total number of tweets loaded from Mongo : 7866


,Created At,Likes,Location,Name,Retweet,Tweet,Tweet_cleaned,_id
0,2019-03-24 01:45:07,0,India,Amar Ujala,0,https://t.co/5n9MTfwWtw\n@BJP4India @INCIndia ...,BJP4India INCIndia #ElectionCommission #Electi...,5c96e12a5c284ca44cf2d1d9
1,2019-03-24 01:45:22,0,"Ohio, USA",JP,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,#DMK #Election2019,5c96e1395c284ca44cf2d1da
2,2019-03-24 01:45:54,0,None,BenarNews Thai,0,พลเอกประยุทธ์ จันทร์โอชา หย่อนบัตรลงคะแนน ที่เ...,10 24 2562 # 2562 # # #bangkok #thailand #elec...,5c96e15a5c284ca44cf2d1db
3,2019-03-24 01:46:11,0,"Tirunelveli, India",கோபிராஜ்.க - GopiRaj.K,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,#DMK #Election2019,5c96e16b5c284ca44cf2d1dc
4,2019-03-24 01:46:25,0,"New Delhi, Delhi",Paresh Kumar,0,"Common Indian voter: हे वराह पुत्र, अंजुलीभर ज...",Common Indian voter Corrupt politicians #Elect...,5c96e1795c284ca44cf2d1dd


In [6]:
df_clean_tweets=df_tweets[df_tweets['Tweet_cleaned']!='']
print('Total number of tweets after removing empty ones {}'.format(df_clean_tweets.shape[0]))

Total number of tweets after removing empty ones 7388


In [7]:
df_clean_tweets.drop_duplicates('Tweet_cleaned',inplace=True)
print('Total number of tweets after removing empty ones {}'.format(df_clean_tweets.shape[0]))

Total number of tweets after removing empty ones 3219


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## As we can see, dropping the duplicates almost reduced the count by half. We need to further explore the data and see if there are any other discrepancies still present

In [8]:
df_clean_tweets['Tweet_cleaned'].value_counts()

#VoteForGlass #JanaSenaParty #JSPForNewAgePolitics                                                                                                                                                                                                                                                          1
Arrogance of BJP4India amp AmitShah may prove fatal in some constituencies You should not try to sideline senior leaders                                                                                                                                                                                    1
Vs # Election2019 Rajib kumarbera                                                                                                                                                                                                                                                                           1
Jaya Prada joined the BJP Azam Khan prepared a lot of water for the sake of drowning BJP4India

In [9]:
df_clean_tweets

,Created At,Likes,Location,Name,Retweet,Tweet,Tweet_cleaned,_id
0,2019-03-24 01:45:07,0,India,Amar Ujala,0,https://t.co/5n9MTfwWtw\n@BJP4India @INCIndia ...,BJP4India INCIndia #ElectionCommission #Electi...,5c96e12a5c284ca44cf2d1d9
1,2019-03-24 01:45:22,0,"Ohio, USA",JP,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,#DMK #Election2019,5c96e1395c284ca44cf2d1da
2,2019-03-24 01:45:54,0,None,BenarNews Thai,0,พลเอกประยุทธ์ จันทร์โอชา หย่อนบัตรลงคะแนน ที่เ...,10 24 2562 # 2562 # # #bangkok #thailand #elec...,5c96e15a5c284ca44cf2d1db
4,2019-03-24 01:46:25,0,"New Delhi, Delhi",Paresh Kumar,0,"Common Indian voter: हे वराह पुत्र, अंजुलीभर ज...",Common Indian voter Corrupt politicians #Elect...,5c96e1795c284ca44cf2d1dd
7,2019-03-24 01:46:37,0,On Bunnerong land,sue travers,0,RT @JustDoGood3: This is part of the answer! 👇...,This is part of the answer Printing off and le...,5c96e1845c284ca44cf2d1e0
10,2019-03-24 01:49:34,0,India,kaushal yadav,0,RT @EasternHerald: JKLF banning on Pakistan da...,JKLF banning on Pakistan day right before #Lok...,5c96e2365c284ca44cf2d1e3
11,2019-03-24 01:50:19,0,None,Soup Guy,0,RT @ThanthiTV: நாடாளுமன்ற தேர்தல் - நாம் தமிழர...,#NaamTamilar #Election2019 #,5c96e2635c284ca44cf2d1e4
12,2019-03-24 01:50:22,0,Thailand,\7ใจนุชเป็นฟ้า🧡,0,"RT @Matcha_Phornin: It’s happening now, #Thail...",It s happening now #Thailand #Election2019 we ...,5c96e2655c284ca44cf2d1e5
14,2019-03-24 01:50:35,0,None,Sanook! News,0,3 in 1 สุดๆ #SanookNews #ข่าววันนี้ #เลือกตั้ง...,3 in 1 #SanookNews # # 62 # 2562 ## 2562 #Elec...,5c96e2725c284ca44cf2d1e7
15,2019-03-24 01:50:50,0,"Kelanang, Selangor",Raaj,0,RT @MakkalSelvarTTV: அம்மா மக்கள் முன்னேற்றக் ...,SDPI K K S M,5c96e2815c284ca44cf2d1e8


In [10]:
df_clean_tweets

,Created At,Likes,Location,Name,Retweet,Tweet,Tweet_cleaned,_id
0,2019-03-24 01:45:07,0,India,Amar Ujala,0,https://t.co/5n9MTfwWtw\n@BJP4India @INCIndia ...,BJP4India INCIndia #ElectionCommission #Electi...,5c96e12a5c284ca44cf2d1d9
1,2019-03-24 01:45:22,0,"Ohio, USA",JP,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,#DMK #Election2019,5c96e1395c284ca44cf2d1da
2,2019-03-24 01:45:54,0,None,BenarNews Thai,0,พลเอกประยุทธ์ จันทร์โอชา หย่อนบัตรลงคะแนน ที่เ...,10 24 2562 # 2562 # # #bangkok #thailand #elec...,5c96e15a5c284ca44cf2d1db
4,2019-03-24 01:46:25,0,"New Delhi, Delhi",Paresh Kumar,0,"Common Indian voter: हे वराह पुत्र, अंजुलीभर ज...",Common Indian voter Corrupt politicians #Elect...,5c96e1795c284ca44cf2d1dd
7,2019-03-24 01:46:37,0,On Bunnerong land,sue travers,0,RT @JustDoGood3: This is part of the answer! 👇...,This is part of the answer Printing off and le...,5c96e1845c284ca44cf2d1e0
10,2019-03-24 01:49:34,0,India,kaushal yadav,0,RT @EasternHerald: JKLF banning on Pakistan da...,JKLF banning on Pakistan day right before #Lok...,5c96e2365c284ca44cf2d1e3
11,2019-03-24 01:50:19,0,None,Soup Guy,0,RT @ThanthiTV: நாடாளுமன்ற தேர்தல் - நாம் தமிழர...,#NaamTamilar #Election2019 #,5c96e2635c284ca44cf2d1e4
12,2019-03-24 01:50:22,0,Thailand,\7ใจนุชเป็นฟ้า🧡,0,"RT @Matcha_Phornin: It’s happening now, #Thail...",It s happening now #Thailand #Election2019 we ...,5c96e2655c284ca44cf2d1e5
14,2019-03-24 01:50:35,0,None,Sanook! News,0,3 in 1 สุดๆ #SanookNews #ข่าววันนี้ #เลือกตั้ง...,3 in 1 #SanookNews # # 62 # 2562 ## 2562 #Elec...,5c96e2725c284ca44cf2d1e7
15,2019-03-24 01:50:50,0,"Kelanang, Selangor",Raaj,0,RT @MakkalSelvarTTV: அம்மா மக்கள் முன்னேற்றக் ...,SDPI K K S M,5c96e2815c284ca44cf2d1e8
